In [ ]:
import meep as mp
import numpy as np
from matplotlib import pyplot as plt
import pp
from pp.sp.meep.add_monitors import add_monitors

In [ ]:
layer_core=1
SOURCE_LAYER=200
PORT1_LAYER=201
PORT2_LAYER=202
res=10
t_oxide=1.0
t_Si=0.22
t_air=0.78
dpml=1
si_zmin=0
clad_material=mp.Medium(epsilon=2.25)
core_material=mp.Medium(epsilon=12)
wavelength=1.55
dfcen=0.2
ymargin=3
xmargin=0
three_d=False
run=False
until_after_sources=100

c = pp.c.waveguide(length=2)
c = add_monitors(c)
component=c
c = pp.extend_ports(component, length=2 * dpml)
c.flatten()
c.x = 0
c.y = 0
pp.show(c)

gdspath = pp.write_gds(c)

fcen = 1 / wavelength
df = dfcen * fcen
cell_thickness = dpml + t_oxide + t_Si + t_air + dpml

cell_zmax = 0.5 * cell_thickness if three_d else 0
cell_zmin = -0.5 * cell_thickness if three_d else 0

si_zmax = 0.5 * t_Si if three_d else 10
si_zmin = -0.5 * t_Si if three_d else -10

geometry = mp.get_GDSII_prisms(core_material, gdspath, layer_core, si_zmin, si_zmax)
cell = mp.GDSII_vol(gdspath, layer_core, cell_zmin, cell_zmax)
cell.size = mp.Vector3(c.xsize - dpml + 2 * xmargin, cell.size[1], cell.size[2])
cell.size += 2 * mp.Vector3(y=ymargin)

src_vol = mp.GDSII_vol(gdspath, SOURCE_LAYER, si_zmin, si_zmax)

p1 = mp.GDSII_vol(gdspath, PORT1_LAYER, si_zmin, si_zmax)
p2 = mp.GDSII_vol(gdspath, PORT2_LAYER, si_zmin, si_zmax)

if three_d:
    oxide_center = mp.Vector3(z=-0.5 * t_oxide)
    oxide_size = mp.Vector3(cell.size.x, cell.size.y, t_oxide)
    oxide_layer = [
        mp.Block(material=clad_material, center=oxide_center, size=oxide_size)
    ]
    geometry = geometry + oxide_layer

sources = [
    mp.EigenModeSource(
        src=mp.GaussianSource(fcen, fwidth=df),
        size=src_vol.size,
        center=src_vol.center,
        eig_band=1,
        eig_parity=mp.NO_PARITY if three_d else mp.EVEN_Y + mp.ODD_Z,
        eig_match_freq=True,
    )
]


In [ ]:
sim = mp.Simulation(
    resolution=res,
    cell_size=cell.size,
    boundary_layers=[mp.PML(dpml)],
    sources=sources,
    geometry=geometry,
)

length = 2
cell_size =cell.size

# Add the mode monitors
fcen = 1/1.55 # 1.55 microns
df = 0.1*fcen # 10% bandwidth
nf = 30 # number of freqs
m1 = sim.add_mode_monitor(fcen,df,nf,mp.FluxRegion(center=[-length/2,0,0],size=[0,cell_size.y,cell_size.z]))
m2 = sim.add_mode_monitor(fcen,df,nf,mp.FluxRegion(center=[length/2,0,0],size=[0,cell_size.y,cell_size.z]))


# Visualize fields and geometry
sim.run(until=100)
plt.figure()
sim.plot2D(fields=mp.Ez)
plt.title('Ez')
plt.savefig('fields.png')

In [ ]:
S12_mode1

In [ ]:
# Run the rest of the simulation
sim.run(until=600)

# Calculate the scattering params for each waveguide
bands = [1,2,3] # just look at first, second, and third, TE modes
m1_results = sim.get_eigenmode_coefficients(m1,[1],eig_parity=(mp.ODD_Z+mp.EVEN_Y)).alpha
m2_results = sim.get_eigenmode_coefficients(m2,bands,eig_parity=(mp.ODD_Z+mp.EVEN_Y)).alpha

a1 = m1_results[:,:,0] #forward wave
b1 = m1_results[:,:,1] #backward wave
a2 = m2_results[:,:,0] #forward wave
b2 = m2_results[:,:,1] #backward wave

S12_mode1 = a2[0,:] / a1[0,:]
S12_mode2 = a2[1,:] / a1[0,:]
S12_mode3 = a2[2,:] / a1[0,:]

freqs = np.array(mp.get_flux_freqs(m1))

# visualize results
plt.figure()
plt.semilogy(1/freqs,np.abs(S12_mode1)**2,'-o',label='S12 Input Mode 1 Output Mode 1')
plt.semilogy(1/freqs,np.abs(S12_mode2)**2,'-o',label='S12 Input Mode 1 Output Mode 2')
plt.semilogy(1/freqs,np.abs(S12_mode3)**2,'-o',label='S12 Input Mode 1 Output Mode 3')
plt.ylabel('Power')
plt.xlabel('Wavelength (microns)')
plt.legend()
plt.grid(True)
plt.savefig('Results.png')
plt.show()

In [ ]:
sim = mp.Simulation(
    resolution=res,
    cell_size=cell.size,
    boundary_layers=[mp.PML(dpml)],
    sources=sources,
    geometry=geometry,
)
mode1 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p1))
mode2 = sim.add_mode_monitor(fcen, 0, 1, mp.ModeRegion(volume=p2))

r = dict()
if run:
    sim.run(until_after_sources=until_after_sources)

    # S parameters
    p1 = sim.get_eigenmode_coefficients(
        mode1, [1], eig_parity=mp.NO_PARITY if three_d else mp.EVEN_Y + mp.ODD_Z
    ).alpha[0, 0, 0]
    p2 = sim.get_eigenmode_coefficients(
        mode2, [1], eig_parity=mp.NO_PARITY if three_d else mp.EVEN_Y + mp.ODD_Z
    ).alpha[0, 0, 1]

    # transmittance
    t = abs(p2) ** 2 / abs(p1) ** 2

    # S parameters
    a1 = sim.get_eigenmode_coefficients(mode1, [1]).alpha[0, 0, 0] # forward 
    b1 = sim.get_eigenmode_coefficients(mode1, [1]).alpha[0, 0, 1] # backward 
    a2 = sim.get_eigenmode_coefficients(mode2, [1]).alpha[0, 0, 0] # forward 
    b2 = sim.get_eigenmode_coefficients(mode2, [1]).alpha[0, 0, 1] # backward 

    S11 = b1 / a1
    S12 = b1 / a2
    S22 = b2 / a2
    S21 = b2 / a1


    r = dict(S11=S11, S22=S22, S12=S12, S21=S21, t=t)

if not three_d:
    sim.plot2D(
        output_plane=mp.Volume(center=mp.Vector3(), size=cell.size),
        fields=mp.Ez,
        field_parameters={"interpolation": "spline36", "cmap": "RdBu"},
    )
r["sim"] = sim


In [ ]:
# Start with geometry
Si = mp.Medium(index=3.4)
SiO2 = mp.Medium(index=1.44)

thickness = 0.22
width1 = 0.5
width2 = 1
length = 4
span = 6
vertices = [
    mp.Vector3(-length/2-span,width1/2),
    mp.Vector3(-length/2,width1/2),
    mp.Vector3(length/2,width2/2),
    mp.Vector3(length/2+span,width2/2),
    mp.Vector3(length/2+span,-width2/2),
    mp.Vector3(length/2,-width2/2),
    mp.Vector3(-length/2,-width1/2),
    mp.Vector3(-length/2-span,-width1/2)
    ]
geometry = [
    mp.Prism(vertices,height=thickness,material=Si) # taper structure
        ]

# Setup domain
resolution = 20
cell_size = mp.Vector3(12,4,0)
boundary_layers = [mp.PML(1.0)]

# Blast it with TE polarized source. Don't worry about an eigenmode source, 
# since we want to measure multiple modes.
sources = [
    mp.EigenModeSource(
    src=mp.GaussianSource(1/1.55,fwidth=0.1/1.55),
    center=[-length/2 - 2],
    size=[0,cell_size.y,cell_size.y],
    eig_parity=mp.ODD_Z+mp.EVEN_Y
    )
]

# Set up simulation object
sim = mp.Simulation(resolution=resolution,
                        cell_size=cell_size,
                        boundary_layers=boundary_layers,
                        geometry=geometry,
                        default_material=SiO2,
                        sources=sources)

# Add the mode monitors
fcen = 1/1.55 # 1.55 microns
df = 0.1*fcen # 10% bandwidth
nf = 30 # number of freqs
m1 = sim.add_mode_monitor(fcen,df,nf,mp.FluxRegion(center=[-length/2,0,0],size=[0,cell_size.y,cell_size.z]))
m2 = sim.add_mode_monitor(fcen,df,nf,mp.FluxRegion(center=[length/2,0,0],size=[0,cell_size.y,cell_size.z]))


# Visualize fields and geometry
sim.run(until=100)
plt.figure()
sim.plot2D(fields=mp.Ez)
plt.title('Ez')
plt.savefig('fields.png')

# Run the rest of the simulation
sim.run(until=600)

# Calculate the scattering params for each waveguide
bands = [1,2,3] # just look at first, second, and third, TE modes
m1_results = sim.get_eigenmode_coefficients(m1,[1],eig_parity=(mp.ODD_Z+mp.EVEN_Y)).alpha
m2_results = sim.get_eigenmode_coefficients(m2,bands,eig_parity=(mp.ODD_Z+mp.EVEN_Y)).alpha

a1 = m1_results[:,:,0] #forward wave
b1 = m1_results[:,:,1] #backward wave
a2 = m2_results[:,:,0] #forward wave
b2 = m2_results[:,:,1] #backward wave

S12_mode1 = a2[0,:] / a1[0,:]
S12_mode2 = a2[1,:] / a1[0,:]
S12_mode3 = a2[2,:] / a1[0,:]

freqs = np.array(mp.get_flux_freqs(m1))

# visualize results
plt.figure()
plt.semilogy(1/freqs,np.abs(S12_mode1)**2,'-o',label='S12 Input Mode 1 Output Mode 1')
plt.semilogy(1/freqs,np.abs(S12_mode2)**2,'-o',label='S12 Input Mode 1 Output Mode 2')
plt.semilogy(1/freqs,np.abs(S12_mode3)**2,'-o',label='S12 Input Mode 1 Output Mode 3')
plt.ylabel('Power')
plt.xlabel('Wavelength (microns)')
plt.legend()
plt.grid(True)
plt.savefig('Results.png')
plt.show()
